<a href="https://colab.research.google.com/github/miiiingi/dacon_sentiment/blob/main/dacon_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive/')
my_folder = '/gdrive/MyDrive/ColabNotebooks/dacon_senti/dataset/dataset'

Mounted at /gdrive/


In [2]:
import pandas as pd
import numpy as np
!pip install konlpy
from konlpy.tag import Mecab
from konlpy.tag import Okt
from tqdm import tqdm
from torchtext.legacy import data, datasets
import torch
import random
import os
import math
from torch import nn as nn
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
SEED = 722
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(SEED)

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 48.2 MB/s 


Torchtext - Field 정의

In [69]:
tokenizer = Okt()
ID = data.Field(sequential=False, use_vocab=False)
TEXT = data.Field(sequential=True, use_vocab=True, tokenize=tokenizer.morphs)
LABEL = data.LabelField(sequential=False, use_vocab=False, is_target=True)

첫 번째 실험 : tokenizer을 morphs(75.37 / clip grad norm x > 과적합 심하게 발생) > nouns(68.4)로 변화 / morphs \\
두 번째 실험 : clip grad norm(73.9 > 과적합 발생) \\
세 번째 실험 : GRU안에 임베딩으로 transformer를 사용하였음 \\
넷 : Adam(0.78) / AdamW(80) \\
다섯 : AdamW(81.11) / AdamW restart(100/10) 80.27 / AdamW restart(100/25) 80.11 \\
여섯 : 특수기호 없앤 것(80.33?) 특수기호 안 없앤 것 : 81.11 \\
일곱 : 초기 가중치 바꾸기 안 바꾸기(승) \\
여덟 : 트랜스포머 헤드 8(81.11)(승) > 1 \\
아홉 : 트랜스포머 임베딩 레이어 128(승) > 512 128(승)> 32(80.33) \\
열 : GRU레이어 선형층 한개 > 두개


Torchtext - 데이터셋 불러와서 정의2 >> 특수문자 없애기

In [87]:
train_data = pd.read_csv(f'{my_folder}/train.csv')
print(train_data)
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","")
print(train_data)
test_data = pd.read_csv(f'{my_folder}/test.csv')
print(test_data)
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","")
print(test_data)
train_data.to_csv(f'{my_folder}/train_rm_punc.csv', index = False)
test_data.to_csv(f'{my_folder}/test_rm_punc.csv', index = False)

        id                                 document  label
0        1                영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐      0
1        2             히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...      1
2        3     괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~      1
3        4              아무래도 20년도지난작품이라 지금보기는너무유치하다      0
4        5  지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.      0
...    ...                                      ...    ...
4995  4996                 좋은 배우들로 3류영화를 찍은 안타까운 영화      0
4996  4997   진짜 드럽게 재미없다 에드워드 호퍼 그림에 배경 빼고는 볼게 아닌영화      0
4997  4998                가장 실망스러운 영화.. 지금까지 본영화중..      0
4998  4999          이런 평점 테러, 네이버에서 좀 막아야 하는 것 아닌가?      1
4999  5000      여주인공이 전작에서는 귀여웠는데, 여기선 완전 망했네, 실망이다      0

[5000 rows x 3 columns]
        id                                document  label
0        1               영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐      0
1        2               히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯      1
2        3        괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요      1
3        4             아무래도 20년도지난작

Torchtext - 데이터셋 불러와서 정의

In [88]:
print(pd.read_csv(f'{my_folder}/train.csv'))
print(pd.read_csv(f'{my_folder}/train_rm_punc.csv'))

        id                                 document  label
0        1                영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐      0
1        2             히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...      1
2        3     괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~      1
3        4              아무래도 20년도지난작품이라 지금보기는너무유치하다      0
4        5  지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.      0
...    ...                                      ...    ...
4995  4996                 좋은 배우들로 3류영화를 찍은 안타까운 영화      0
4996  4997   진짜 드럽게 재미없다 에드워드 호퍼 그림에 배경 빼고는 볼게 아닌영화      0
4997  4998                가장 실망스러운 영화.. 지금까지 본영화중..      0
4998  4999          이런 평점 테러, 네이버에서 좀 막아야 하는 것 아닌가?      1
4999  5000      여주인공이 전작에서는 귀여웠는데, 여기선 완전 망했네, 실망이다      0

[5000 rows x 3 columns]
        id                                document  label
0        1               영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐      0
1        2               히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯      1
2        3        괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요      1
3        4             아무래도 20년도지난작

In [124]:
train_data = data.TabularDataset(
                            path = f'{my_folder}/train.csv',
                            format = 'csv',
                            fields = [('id', ID), ('document', TEXT), ('label', LABEL)],
                            skip_header = True,)
test_data = data.TabularDataset(
                            path = f'{my_folder}/test.csv',
                            format = 'csv',
                            fields = [('id', ID), ('document', TEXT)],
                            skip_header = True,)
# train_data = data.TabularDataset(
#                             path = f'{my_folder}/train_rm_punc.csv',
#                             format = 'csv',
#                             fields = [('id', ID), ('document', TEXT), ('label', LABEL)],
#                             skip_header = True,)
# test_data = data.TabularDataset(
#                             path = f'{my_folder}/test_rm_punc.csv',
#                             format = 'csv',
#                             fields = [('id', ID), ('document', TEXT)],
#                             skip_header = True,)

train_data, valid_data = train_data.split(split_ratio = 0.5)

Torchtext - 단어 집합 만들기

In [125]:
TEXT.build_vocab(train_data, min_freq=5, max_size= 1000) # 단어 집합 생성
# LABEL.build_vocab(train_data)

In [126]:
vars(train_data[1])

{'document': ['우리나라', '좀비', '영화', '는', '왜', '이러냐', '..'],
 'id': '1494',
 'label': '0'}

Torchtext - 배치화 시키기

In [127]:
batch_size = 64
train_loader = data.Iterator(train_data, batch_size=batch_size, shuffle=True)
val_loader = data.Iterator(valid_data, batch_size=batch_size, train=False, sort=False)
test_loader = data.Iterator(test_data, batch_size=batch_size, train=False, sort=False)

In [128]:
print(next(iter(train_loader)))
print(next(iter(val_loader)))
print(next(iter(test_loader)))


[torchtext.legacy.data.batch.Batch of size 64]
	[.id]:[torch.LongTensor of size 64]
	[.document]:[torch.LongTensor of size 21x64]
	[.label]:[torch.LongTensor of size 64]

[torchtext.legacy.data.batch.Batch of size 64]
	[.id]:[torch.LongTensor of size 64]
	[.document]:[torch.LongTensor of size 21x64]
	[.label]:[torch.LongTensor of size 64]

[torchtext.legacy.data.batch.Batch of size 64]
	[.id]:[torch.LongTensor of size 64]
	[.document]:[torch.LongTensor of size 19x64]


Embedding layer 어떻게 처리할 지 생각해보자

Model 정의

In [137]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        even_div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        odd_div_term = torch.exp(torch.arange(1, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * even_div_term)
        pe[:, 1::2] = torch.cos(position * odd_div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        pe.requires_grad = False
        self.register_buffer('pe', pe)
    def forward(self, x):
        return x + self.pe[:x.size(0), :]

class Transformer(nn.Module):
    def __init__(self, n_vocab, n_embed_pre, n_embed_post, n_layers, n_head, dropout):
        super(Transformer, self).__init__()
        self.src_mask = None
        self.embed = nn.Embedding(n_vocab, n_embed_post)
        self.pos_enc = PositionalEncoding(n_embed_post)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=n_embed_post, nhead = n_head, dropout=dropout)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=n_layers)        
        self.n_embed_post = n_embed_post
        self.init_weights()

    def init_weights(self) -> None:
        nn.init.xavier_normal_(self.embed.weight)
    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            self.src_mask = self._generate_square_subsequent_mask(src.shape[0]).to(src.device)
        src = self.embed(src)
        src = self.pos_enc(src)
        output = self.encoder(src, self.src_mask)
        return output

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [180]:
import torch.nn.functional as F
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, model, dropout=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.embed = model
        self.dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers * 2,
                          bidirectional = True)
        self.out = nn.Linear(self.hidden_dim * 2, n_classes)

    def forward(self, x):
        x = self.embed(x)
        x, _ = self.gru(x)  # GRU의 리턴값은 (시퀀스 길이, 배치 크기, 은닉 상태의 크기)
        h_t = x[-1, :, :] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [186]:
Epochs = 300
lr = 0.0001
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transformer = Transformer(n_vocab = len(TEXT.vocab),n_embed_pre = 25, n_embed_post = 128, n_layers = 1, n_head = 8, dropout = 0).to(DEVICE)
model = GRU(n_layers = 1, hidden_dim = 256, n_vocab = len(TEXT.vocab), embed_dim = 128, n_classes = 2, model = transformer, dropout = 0.3).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr= lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0= Epochs // 10, T_mult=2, eta_min = lr)

In [187]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.document.to(DEVICE), batch.label.long().to(DEVICE)
        # print(x.shape)
        # print(model(x).shape)
        # print(y.shape)
        logit = model(x)
        # print(logit.shape)
        # print(y.shape)
        loss = F.cross_entropy(logit, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # scheduler.step()

In [188]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    with torch.no_grad():
      for batch in val_iter:
          x, y = batch.document.to(DEVICE), batch.label.long().to(DEVICE)
          logit = model(x)
          loss = F.cross_entropy(logit, y)
          total_loss += loss.item()
          corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
      size = len(val_iter.dataset)
      avg_loss = total_loss / size
      avg_accuracy = 100.0 * corrects / size
      return avg_loss, avg_accuracy

In [189]:
def evaluate_test(model, test_loader, device):
    result = []
    model.to(device)
    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_loader):
            x = batch.document.to(device)
            out = model(x)
            pred = torch.argmax(out, dim=1)
            result.append(pred)
    return torch.cat(result).cpu().numpy()

In [190]:
best_val_loss = None
val_accuracy_accum = list()

for e in tqdm(range(Epochs)):
    train(model, optimizer, train_loader)
    train_loss, train_accuracy = evaluate(model, train_loader)
    val_loss, val_accuracy = evaluate(model, val_loader)

    print("[Epoch: %d] train loss : %5.2f | train acc : %5.2f | val loss : %5.2f | val accuracy : %5.2f" % (e+1, train_loss, train_accuracy, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot"):
            os.makedirs("/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot")
        torch.save(model.state_dict(), f'/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot/sentiment.pt')
        best_val_loss = val_accuracy
        val_accuracy_accum.append(best_val_loss)
print(f'final validation score : {max(val_accuracy_accum)}')

  0%|          | 1/300 [00:03<17:02,  3.42s/it]

[Epoch: 1] train loss :  0.01 | train acc : 48.80 | val loss :  0.01 | val accuracy : 48.64


  1%|          | 2/300 [00:06<16:30,  3.32s/it]

[Epoch: 2] train loss :  0.01 | train acc : 51.20 | val loss :  0.01 | val accuracy : 51.36


  1%|          | 3/300 [00:09<16:02,  3.24s/it]

...

[Epoch: 131] train loss :  0.00 | train acc : 99.64 | val loss :  0.02 | val accuracy : 76.64


 44%|████▍     | 132/300 [06:43<08:32,  3.05s/it]

[Epoch: 132] train loss :  0.00 | train acc : 99.52 | val loss :  0.02 | val accuracy : 77.04


 44%|████▍     | 132/300 [06:45<08:36,  3.07s/it]


KeyboardInterrupt: ignored

검증 데이터셋 결과 살펴보기

In [16]:
model.load_state_dict(torch.load(f'/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot/sentiment.pt'))
result = evaluate_test(model, val_loader, DEVICE)

100%|██████████| 40/40 [00:00<00:00, 55.26it/s]


In [27]:
import collections
inference = collections.defaultdict(int)
for sentence, label in zip(valid_data, result) :
  dicts = vars(sentence)['document']
  answer = vars(sentence)['label']
  if int(answer) != label :
    print(f'document : {dicts}, prediction : {label}, answer : {answer}')    

document : ['평점', '왜캐', '낮아', '.', '공포', '영화', '톱', '10', '안', '에', '들어감', 'ㄹㅇ', '개', '무서움'], prediction : 0, answer : 1
document : ['할머니', '=', '나', '랑', '같이', '농구', '해', '보지', '않겠나', '?', '세계', '를', '노리자꾸나'], prediction : 1, answer : 0
document : ['1/3', '정도', '보다', '말았는데', '예상', '되는', '그', '결말', '일', '듯'], prediction : 1, answer : 0
document : ['굶주린', '총각', '이', '재미있게', '볼수', '있을', '법', '한', '영화'], prediction : 1, answer : 0
document : ['푸하하하', '하하', 'ㅏㅎ', '하', 'ㅏㅎ', '머리', '봐', 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '좀', '짱', '인', '듯', '??', 'ㅋㅋㅋ'], prediction : 1, answer : 0
document : ['졸고', '또', '졸고', '...', '리즈', '왜', '이런', '영화', '를', '고른거야', 'ㅠㅠ'], prediction : 1, answer : 0
document : ['캐릭터', '의', '매력', '조차', '있는듯', '없는듯', '...'], prediction : 1, answer : 0
document : ['이', '거', '미국', '판', '가면라이더', '디케이드', '네'], prediction : 1, answer : 0
document : ['아', '..', '그냥', '이', '영화', '를', '보는', '동안', '인생', '은', '참', '...', '암울한거', '같아'], prediction : 1, answer : 0
document : ['영화', '예고편', '이', '제일', '무서

검증 데이터셋에 대해서 예측한 결과와 정답을 살펴보았을 때, 성능이 그다지 좋지 않은 것을 알 수 있다.다르게 예측한 경우의 분석(morphs) \\
첫 번째, 비속어는 잘 인지를 못하는 것 같다. \\
두 번째, 부정어 인식이 잘 안되는 것 같다(킬링타임용으로도 보고 싶지 않은 영화 > 왜 긍정으로 인식하지..?) \\
세 번째, 단어의 가중치가 잘못 매겨져 있는 느낌(야밤에 완전 깔깔 대고 웃었다 정신 없고 재밌음 > 왜 부정으로 인식하지..?) \\
일단 우선적으로 특수문자를 제외해보자 >> 처음으로 돌아가자.

테스트 데이터셋 결과 살펴보기

In [191]:
model.load_state_dict(torch.load(f'/gdrive/MyDrive/ColabNotebooks/dacon_senti/snapshot/sentiment.pt'))
result = evaluate_test(model, test_loader, DEVICE)

100%|██████████| 79/79 [00:01<00:00, 57.81it/s]


In [192]:
import collections
inference = collections.defaultdict(int)
for sentence, label in zip(test_data, result) :
  dicts = vars(sentence)['document']
  print(f'document : {dicts}, label : {label}')

document : ['시간', '때우기', '좋은', '영화', '지루함'], label : 1
document : ['훈훈한', '정이', '느껴지는', '영화', '!', '가족', '끼리', '드라마', '보듯이', '보면', '딱', '~!'], label : 1
document : ['Childhood', 'fantasy'], label : 1
document : ['멋있는', '영화', '입니다', '.', '잊을', '수', '없는', '!'], label : 1
document : ['너무', '감동', '적이네요', '펑펑', '울었습니다'], label : 0
document : ['어이', '김구라', '자세', '좀', '똑바로', '앉어', '방송', '마다', '왜케', '삐', '딱하게', '앉는거지'], label : 0
document : ['친구', '한테', '추천', '받고', '본', '영화', '이지만', '내', '가', '남', '들', '한테', '추천', '하는', '영화'], label : 0
document : ['머', '내용', '은', '그닥', '...', '특별하지', '않다'], label : 0
document : ['강', '배우', '엄청난데', '...', '여군', '버전', '으로', '다시', '안', '찍', '나'], label : 0
document : ['이렇게', '재밋게본', '드라마', '추노말거', '없엇음', 'ㅜ', '짱'], label : 1
document : ['나가토', '의', '나가토', '에', '의한', '나가토', '를', '위', '한'], label : 0
document : ['나', '만', '당할수', '없다', '이', '걸', '영화', '라고', '시간', '낭비'], label : 0
document : ['헬드', '핸드', '기법', '의', '대표', '적', '인', '실패', '작'], label : 1
document : ['

In [193]:
submission = pd.read_csv(f"{my_folder}/sample_submission.csv")
submission["label"] = result
print(submission)
submission.to_csv(f"{my_folder}/submission.csv", index=False)

        id  label
0        1      1
1        2      1
2        3      1
3        4      1
4        5      0
...    ...    ...
4995  4996      0
4996  4997      0
4997  4998      1
4998  4999      0
4999  5000      0

[5000 rows x 2 columns]
